随便写的

In [ ]:
import matplotlib.pyplot as plt

epochs = [0, 1, 2, 3]
train_losses = [1.2939, 1.2912, 1.1544, 1.1017]
lrs = [0.010000, 0.007719, 0.005359, 0.002872]
dice_coefficients = [0.226, 0.226, 0.228, 0.099]
mean_ious = [6.4, 6.4, 8.9, 42.8]

# 折线图设置
plt.figure(figsize=(10, 6))  # 调整图片大小

plt.subplot(2, 1, 1)  # 上方子图：训练指标
plt.plot(epochs, train_losses, label='Train Loss', marker='o')
plt.plot(epochs, lrs, label='Learning Rate', marker='o')
plt.plot(epochs, dice_coefficients, label='Dice Coefficient', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Values')
plt.title('Training Metrics')
plt.legend()
plt.grid(True)

# 条形图设置
plt.subplot(2, 1, 2)  # 下方子图：IoU
bar_labels = ['Class 0', 'Class 1']  # 标签
iou_by_epoch = [[0.0, 12.7], [5.0, 12.9], [77.4, 8.1]]  # 每轮的 IoU 数据

for i in range(len(epochs)):
    plt.bar(bar_labels, iou_by_epoch[i], label=f'Epoch {i}', bottom=mean_ious[i])

plt.xlabel('Class')
plt.ylabel('IoU')
plt.title('Intersection over Union (IoU)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
! pip install wandb

In [ ]:
import logging

logger = logging.getLogger("wandb")
logger.setLevel(logging.WARNING)

In [ ]:
# This is secret and shouldn't be checked into version control
WANDB_API_KEY=17acd1bc31a78695a83ccc49a28bee44d314e55e
# Name and notes optional
WANDB_NAME="My first run"
WANDB_NOTES="Smaller learning rate, more regularization."

In [ ]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()